In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [788 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRele

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17747349|R2EI7QLPK4LF7U|B00U7LCE6A|     106182406|CCleaner Free [Do...|Digital_Software|          4|            0|          0|   N|                Y|          Four Stars|      So far so good| 2015-08-31|
|         US|   10956619|R1W5OMFK1Q3I3O|B00HRJMOM4|     162269768|ResumeMaker Profe...|Digital_Software|          3|    

In [5]:
# Create a DataFrame where total_votes count is equal to or greater than 20, to avoid having division by zero errors later
df_1 = df.filter(df.total_votes > 20)
df_1.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53081365|R10Q403A12Q9PX|B00CS75YKE|     886225685|Adobe Premiere Pr...|Digital_Software|          1|          123|        136|   N|                N|Very sad that Ado...|Very sad that Ado...| 2015-08-29|
|         US|   40439671|R2OH6106YSNSV4|B00OVT6VKY|     783437457|Excel Budget Soft...|Digital_Software|          1|    

In [6]:
# Filter df_1 to retrieve rows where helpful_votes / total_votes is greater or equal than 50%
df_2 = df_1.filter((df_1.helpful_votes/df_1.total_votes)>=0.5)
df_2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53081365|R10Q403A12Q9PX|B00CS75YKE|     886225685|Adobe Premiere Pr...|Digital_Software|          1|          123|        136|   N|                N|Very sad that Ado...|Very sad that Ado...| 2015-08-29|
|         US|   40439671|R2OH6106YSNSV4|B00OVT6VKY|     783437457|Excel Budget Soft...|Digital_Software|          1|    

In [7]:
# Filter df_2 to retrieve all the rows where a review was written as part of the Vine program
df_paid = df_2.filter(df_2.vine == "Y")
df_paid.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [8]:
# Filter df_2 to retrieve all the rows where the review was not part of the Vine program
df_unpaid = df_2.filter(df_2.vine == "N")
df_unpaid.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53081365|R10Q403A12Q9PX|B00CS75YKE|     886225685|Adobe Premiere Pr...|Digital_Software|          1|          123|        136|   N|                N|Very sad that Ado...|Very sad that Ado...| 2015-08-29|
|         US|   40439671|R2OH6106YSNSV4|B00OVT6VKY|     783437457|Excel Budget Soft...|Digital_Software|          1|    

# Summary

In [9]:
# Total number of paid reviews
total_paid_reviews = df_paid.count()
print(f"Number of paid reviews: {total_paid_reviews}")

Number of paid reviews: 0


In [10]:
# Paid 5-star reviews
paid_five_star_reviews = df_paid.filter(df_paid.star_rating == 5).count()
print(f"Paid 5-star reviews: {paid_five_star_reviews}")

Paid 5-star reviews: 0


In [15]:
# Percentage of paid 5-star reviews
try: (paid_five_star_reviews / total_paid_reviews) * 100

except: print("There are no paid reviews for Digital Software Purchases")

There are no paid reviews for Digital Software Purchases


In [12]:
# Total number of unpaid reviews
total_unpaid_reviews = df_unpaid.count()
print(f"Number of unpaid reviews: {total_unpaid_reviews}")

Number of unpaid reviews: 2011


In [13]:
# Unpaid 5-star reviews
unpaid_five_star_reviews = df_unpaid.filter(df_unpaid.star_rating == 5).count()
print(f"Unpaid 5-star reviews: {unpaid_five_star_reviews}")

Unpaid 5-star reviews: 300


In [19]:
# Percentage of unpaid 5-star reviews
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100
print(f"Percentage of total reviews: {unpaid_five_star_percent:.4g}%")

Percentage of total reviews: 14.92%
